In [3]:
import numpy as np
import pandas as pd

# from mapping import mp

original_data_file = '../datasets/pid.xlsx'
xl = pd.ExcelFile(original_data_file)
print(xl.sheet_names)

['Quy tắc phân loại', 'T1.2020', '2019', '2018', 'T1-T3.2017', '2016', '2015', '2013', 'Summary']


In [ ]:
mp = {
    "SEA": "SEA",
    "A32431": "SEA",
    "3.7": "A37",
    "A32428": "A37",
    "4.2": "A42",
    "A32426": "A42",
    "THAI": "THAI",
    "A32432": "THAI",
    "FIL": "FIL",
    "HbCS": "HBCS",
    "A32427": "HBCS",
    "HbQs": "HBQS",
    "A32441": "HBQS",
    "2delT": "C2DELT",
    "A32429": "C2DELT",
    
    "-28": "28",
    "cd 27/28": "28",
    "CD28": "28",
    "cd15": "CD15",
    "cd 17": "CD17",
    "CD17": "CD17",
    "A32435": "CD17",
    "Co17": "CD17",
    "cd 19": "CD19",
    "cd 8/9": "CD8.9",
    "cd 26": "CD26",
    "CD26": "CD26",
    "Co26": "CD26",
    "A32433": "CD26",
    "41/42": "CD41.42",
    "A32442": "CD41.42",
    "cd 43": "CD43",
    "71/72": "CD71.72",
    "cd 95": "CD95",
    "CD95": "CD95",
    "cd121": "CD121",
    "IVS": "IVS1.1",
    "A32440": "IVS2.654",
    "89/90": "90",
    "cd90": "90",
    "-88": "88",
    "b-29": "29",
    "b-31": "31"
}

In [7]:
df1 = xl.parse(xl.sheet_names[1])
df2 = xl.parse(xl.sheet_names[2])
df3 = xl.parse(xl.sheet_names[3])
df4 = xl.parse(xl.sheet_names[4])
df5 = xl.parse(xl.sheet_names[5])
df6 = xl.parse(xl.sheet_names[6])
df7 = xl.parse(xl.sheet_names[7])

In [10]:
df2['Xét nghi?m']


0                                         HbCS:DUONG TÍNH
1                                       431SEA:DUONG TÍNH
2       b-cd 41/42:DUONG TÍNH, b-cd71/72:ÂM TÍNH, b-cd...
3                                    JAK2V617F:DUONG TÍNH
4                                    JAK2V617F:DUONG TÍNH
                              ...                        
6289                                 JAK2V617F:DUONG TÍNH
6290                         THAI:ÂM TÍNH, 431SEA:ÂM TÍNH
6291                                    JAK2V617F:ÂM TÍNH
6292    HbCS:ÂM TÍNH, a3.7:ÂM TÍNH, 431SEA:ÂM TÍNH, TH...
6293                                      c.2delT:ÂM TÍNH
Name: Xét nghi?m, Length: 6294, dtype: object